In [37]:
import sys
from pathlib import Path
import os

def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False

def clone_repository() -> None:
    !git clone https://github.com/featurestorebook/mlfs-book.git
    %cd mlfs-book

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml

if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    # Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
    if root_dir.parts[-1:] == ('airquality',):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ('notebooks',):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir) 
    print("Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    sys.path.append(root_dir)
print(f"Added the following directory to the PYTHONPATH: {root_dir}")
    
# Set the environment variables from the file <root_dir>/.env
from mlfs import config
if os.path.exists(f"{root_dir}/.env"):
    settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

Local environment
Added the following directory to the PYTHONPATH: /home/olive/dev/lab1A
HopsworksSettings initialized!


<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [38]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from mlfs.airquality import util
from mlfs import config
import json
import warnings
warnings.filterwarnings("ignore")

## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [39]:
def get_info_hopsworks(street):
    project = hopsworks.login()
    fs = project.get_feature_store() 
    secrets = hopsworks.get_secrets_api()
    
    # This line will fail if you have not registered the AQICN_API_KEY as a secret in Hopsworks
    AQICN_API_KEY = secrets.get_secret("AQICN_API_KEY").value
    location_str = secrets.get_secret(f"SENSOR_LOCATION_JSON_{street.lower()}").value
    location = json.loads(location_str)
    
    country=location['country']
    city=location['city']
    street=location['street']
    aqicn_url=location['aqicn_url']
    latitude=location['latitude']
    longitude=location['longitude']
    
    today = datetime.date.today()
    
    location_str
    return project, fs, AQICN_API_KEY, country, city, street, aqicn_url, latitude, longitude, today

### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [40]:
# Retrieve feature groups
def fetch_fgs(street, fs):
    air_quality_fg = fs.get_feature_group(
        name=f"air_quality_v_agrade_{street.lower()}",
        version=1,
    )
    weather_fg = fs.get_feature_group(
        name=f"weather_v_agrade_{street.lower()}",
        version=1,
    )
    return air_quality_fg, weather_fg

---

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


In [41]:
def retrieve_aq_today(aqicn_url, country, city, street, today, AQICN_API_KEY):
    import requests
    import pandas as pd
    
    aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQICN_API_KEY)
    aq_today_df
    
    return aq_today_df

In [42]:
# NEW. NEED TO ADD ROLLING FROM PREVIOUS TO THIS DAY
# FETCH LAST THREE PM25s FROM df
def add_rollings_today(air_quality_fg,today,aq_today_df):
    import time
    #query = air_quality_fg.select_all().filter(air_quality_fg["date"] >= datetime.date.today()-datetime.timedelta(days=4))
    
    #df_recent = query.read()
    #print(df_recent)
    #aq_today_df["roll"] = df_recent["pm25"].mean().astype("float64")
    #aq_today_df = aq_today_df[["date","pm25","roll","country","city","street","url"]]
    #aq_today_df
    query = air_quality_fg.select_all().filter(air_quality_fg['date']>= today-datetime.timedelta(days=4))
    
    df_recent = query.read()
    
    df_recent = df_recent.sort_values('date')
    print(df_recent)
    pm_prev3 = df_recent['pm25'].head(3)
    aq_today_df['rolling3'] = pm_prev3.mean()
    
    aq_today_df['lag1d'] = df_recent['pm25'].iloc[2]
    aq_today_df['lag2d'] = df_recent['pm25'].iloc[1]
    aq_today_df['lag3d'] = df_recent['pm25'].iloc[0]
    
    
    aq_today_df 
    
    return aq_today_df

In [43]:
#aq_today_df.info()

## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [44]:
def get_weather_forecast(city, latitude, longitude):
    hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
    hourly_df = hourly_df.set_index('date')
    
    # We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
    # We only want the daily weather data, so only get weather at 12:00
    daily_df = hourly_df.between_time('11:59', '12:01')
    daily_df = daily_df.reset_index()
    daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
    daily_df['date'] = pd.to_datetime(daily_df['date'])
    daily_df['city'] = city
    daily_df
    
    return daily_df

In [45]:
#daily_df.info()

## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [46]:
def aq_insert_new_data(air_quality_fg,aq_today_df):
    # Insert new data
    air_quality_fg.insert(aq_today_df)

    return air_quality_fg

In [47]:
# Insert new data
def weather_insert(weather_fg, daily_df):
    weather_fg.insert(daily_df, wait=True)

    return weather_fg

In [48]:
locations = {
                "imahashi":
                    {
                        "url":"https://api.waqi.info/feed/@2543",
                        "lat":"34.766389", 
                        "long":"137.393889"
                    },
                "azuma": 
                    {
                        "url":"https://api.waqi.info/feed/@2374",
                        "lat":"34.7632176",
                        "long":"137.4145777"
                    },
                "osaki": 
                    {
                        "url":"https://api.waqi.info/feed/@2372",
                        "lat":"34.712222",
                        "long":"137.346944"
                    },
                "noyori": 
                    {
                        "url":"https://api.waqi.info/feed/@6600",
                        "lat":"34.698889", 
                        "long":"137.39"
                    },
                "oiwa": 
                    {
                        "url":"https://api.waqi.info/feed/@2373",
                        "lat":"34.721944", 
                        "long":"137.450833"
                    }
            }

def full_pipeline(stations):
    
    for location in stations:

        street = location
    
        project, fs, AQICN_API_KEY, country, city, street, url, latitude, longitude, today = get_info_hopsworks(street)
    
        air_quality_fg, weather_fg = fetch_fgs(street, fs)
        
        aq_today_df = retrieve_aq_today(url, country, city, street, today, AQICN_API_KEY)
    
        aq_today_df = add_rollings_today(air_quality_fg,today,aq_today_df)
    
        daily_df = get_weather_forecast(city, latitude, longitude)
    
        air_quality_fg = aq_insert_new_data(air_quality_fg,aq_today_df)
    
        weather_fg = weather_insert(weather_fg, daily_df)

    
full_pipeline(locations)

2025-11-15 15:05:24,791 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-11-15 15:05:24,793 INFO: Initializing external client
2025-11-15 15:05:24,794 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-11-15 15:05:26,138 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1286329
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.59s) 
                       date  pm25   rolling3  lag1d  lag2d  lag3d country  \
0 2025-11-12 00:00:00+00:00  27.0  25.000000   21.0   23.0   31.0   Japan   
3 2025-11-13 00:00:00+00:00  34.0  23.666666   27.0   21.0   23.0   Japan   
1 2025-11-14 00:00:00+00:00  46.0  27.333334   34.0   27.0   21.0   Japan   
2 2025-11-15 00:00:00+00:00  39.0  35.666668   46.0   34.0   27.0   Japan   

        city    street                               url  
0  Toyohashi  imahashi  https://api.waqi.info/feed/@2543  
3  Toyohashi  imahashi  https://api.waqi.info/feed/@2543  
1  Toyohashi  imahashi  https://api.waqi.info/feed/@2543  
2  Toyohashi  imahashi  https://api.waqi.info/feed/@2543  
Coordinates 34.75°N 137.5°E
Elevation 12.0 m asl
Timezone None None
Timezone difference to GMT+0

Uploading Dataframe: 100.00% |██████████████████████████| Rows 1/1 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: air_quality_v_agrade_imahashi_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1286329/jobs/named/air_quality_v_agrade_imahashi_1_offline_fg_materialization/executions
2025-11-15 15:05:44,942 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1286329/fs/1273951/fg/1703561


Uploading Dataframe: 100.00% |██████████████████████████| Rows 7/7 | Elapsed Time: 00:01 | Remaining Time: 00:00


...

2025-11-15 15:17:42,213 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1286329
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.61s) 
                       date  pm25   rolling3  lag1d  lag2d  lag3d country  \
1 2025-11-12 00:00:00+00:00  28.0  28.333334   25.0   28.0   32.0   Japan   
3 2025-11-13 00:00:00+00:00  39.0  27.000000   28.0   25.0   28.0   Japan   
2 2025-11-14 00:00:00+00:00  47.0  30.666666   39.0   28.0   25.0   Japan   
0 2025-11-15 00:00:00+00:00  41.0  38.000000   47.0   39.0   28.0   Japan   

        city street                               url  
1  Toyohashi   oiwa  https://api.waqi.info/feed/@2373  
3  Toyohashi   oiwa  https://api.waqi.info/feed/@2373  
2  Toyohashi   oiwa  https://api.waqi.info/feed/@2373  
0  Toyohashi   oiwa  https://api.waqi.info/feed/@2373  
Coordinates 34.75°N 137.5°E
Elevation 24.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
2025-11-15

Uploading Dataframe: 100.00% |██████████████████████████| Rows 1/1 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: air_quality_v_agrade_oiwa_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1286329/jobs/named/air_quality_v_agrade_oiwa_1_offline_fg_materialization/executions
2025-11-15 15:18:02,497 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1286329/fs/1273951/fg/1703565


Uploading Dataframe: 100.00% |██████████████████████████| Rows 7/7 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: weather_v_agrade_oiwa_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1286329/jobs/named/weather_v_agrade_oiwa_1_offline_fg_materialization/executions
2025-11-15 15:18:18,291 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-11-15 15:19:09,267 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-11-15 15:20:35,166 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-11-15 15:20:35,321 INFO: Waiting for log aggregation to finish.
2025-11-15 15:20:45,730 INFO: Execution finished successfully.


## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
